In [1]:
import torch
import random
import numpy as np
import pandas as pd
import transformers
from datasets import Dataset
from datasets import DatasetDict

from transformers import AutoTokenizer
from datasets import ClassLabel, Sequence
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split

from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

from transformers import AlbertForTokenClassification, TokenClassificationPipeline, PreTrainedTokenizerFast






/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-14 10:12:00.493994: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 10:12:00.537290: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "sagorsarker/bangla-bert-base"
# model_checkpoint = "51la5/roberta-large-NER"
# model_checkpoint = "FacebookAI/xlm-roberta-large-finetuned-conll03-german"
# model_checkpoint = "neuropark/sahajBERT-NER"
batch_size = 64

In [3]:
import ast
dataset = pd.read_csv('/home/bikas/NER_Jsonl/generated_dataset.csv', converters={'tokens': ast.literal_eval, 'tags': ast.literal_eval})

In [4]:
dataset["tags"][0][3]

'LOC'

In [5]:
dataset["tokens"][0]

['কমিউনিটি কলেজের',
 'শহরগুলির',
 'তসুশিমা দ্বীপ',
 'দক্ষিণে হরিয়ানা উত্তরপ্রদেশ রাজ্য',
 'ফাতিমাহ স্যাহরিনি জায়লানি',
 '08:28:39',
 '07:13:28',
 'মিয়া সেবাস্টিনের']

In [6]:
dataset.shape

(3716, 2)

In [7]:
dataset = dataset.rename(columns={'tags': 'ner_tags'})

In [8]:
 # Split the data into training, validation, and test sets
train, X_temp = train_test_split(dataset, test_size=0.3, random_state=42)
validation, test = train_test_split(X_temp, test_size=0.10, random_state=42)

In [9]:
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)
test = test.reset_index(drop=True)

In [10]:
 # Create a datasets.Dataset object
train = Dataset.from_dict(train)
validation = Dataset.from_dict(validation)
test = Dataset.from_dict(test)
# Print the dataset to check the format
print("Train Dataset : ",train)
print("Validation Dataset : ", validation)
print("Test Dataset : ", test)

Train Dataset :  Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 2601
})
Validation Dataset :  Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 1003
})
Test Dataset :  Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 112
})


In [11]:
# Create DatasetDict
datasets = DatasetDict({
    'validation': validation,
    'test': test,
    'train': train
})
print("Dataset: \n", datasets)

Dataset: 
 DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1003
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 112
    })
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2601
    })
})


In [12]:
datasets["train"]["tokens"][1][4]

'রেচেল রবার্টস'

In [13]:
from datasets import ClassLabel, Sequence

# provide the label_names yourself
# label_names = [...]
# OR get them from the dataset
label_names = sorted(set(label for labels in datasets["train"]["ner_tags"] for label in labels))

# Cast to ClassLabel
datasets = datasets.cast_column("ner_tags", Sequence(ClassLabel(names=label_names)))

Casting the dataset: 100%|██████████| 2601/2601 [00:00<00:00, 22832.96 examples/s]


In [14]:
print(datasets["train"].features[f"ner_tags"])

Sequence(feature=ClassLabel(names=['LOC', 'NUM', 'OBJ', 'ORG', 'PER', 'TIME'], id=None), length=-1, id=None)


In [15]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
print("\nLabel Test: \n", label_list)


Label Test: 
 ['LOC', 'NUM', 'OBJ', 'ORG', 'PER', 'TIME']


In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [17]:
example = datasets["train"][4]
print(example["tokens"][0])

ফেরদৌস বড়লোকের


In [18]:
datasets["train"][4]

{'tokens': ['ফেরদৌস বড়লোকের',
  'বিদ্রোহী গঙ্গানারায়ণ',
  'জাইগীর উচ্চ বিদ্যালয়',
  'এম প্রাথমিক বিদ্যালয়',
  'ভলুপেটাস ডেভেলপার্সের',
  'খালসা কলেজএ',
  '7532',
  'কাগজগুলো'],
 'ner_tags': [4, 4, 3, 3, 4, 3, 1, 2]}

In [19]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '[UNK]', 'বড', '##লে', '##াকে', '##র', 'বিদ', '##রে', '##াহ', '##ী', 'গ', '##ঙ', '##গানা', '##রা', '##য', '##ণ', 'জাইগ', '##ীর', 'উচচ', 'বিদ', '##যা', '##ল', '##য', 'এম', 'পরা', '##থম', '##িক', 'বিদ', '##যা', '##ল', '##য', 'ভল', '##পেটা', '##স', 'ডেভেলপারস', '##ের', 'খালসা', 'কলেজ', '##এ', '[UNK]', 'কাগজ', '##গলে', '##া', '[SEP]']


In [20]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '[UNK]', 'বড', '##লে', '##াকে', '##র', 'বিদ', '##রে', '##াহ', '##ী', 'গ', '##ঙ', '##গানা', '##রা', '##য', '##ণ', 'জাইগ', '##ীর', 'উচচ', 'বিদ', '##যা', '##ল', '##য', 'এম', 'পরা', '##থম', '##িক', 'বিদ', '##যা', '##ল', '##য', 'ভল', '##পেটা', '##স', 'ডেভেলপারস', '##ের', 'খালসা', 'কলেজ', '##এ', '[UNK]', 'কাগজ', '##গলে', '##া', '[SEP]']


In [21]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(8, 44)

In [22]:
print(tokenized_input.word_ids())

[None, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 6, 7, 7, 7, None]


In [23]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

44 44


In [24]:
label_all_tokens = True

In [25]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [26]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 100, 10448, 12041, 32993, 8507, 11604, 23345, 32615, 13168, 2094, 2395, 80304, 100, 4939, 96152, 2046, 15676, 5174, 9305, 56885, 20692, 11604, 4009, 2395, 12351, 3012, 2830, 100, 1040, 100, 1040, 100, 100, 1019, 100, 1019, 100, 102], [101, 2441, 4597, 13399, 39221, 10273, 7724, 8705, 12351, 8357, 2356, 52338, 23206, 3447, 8446, 2343, 8411, 9417, 37674, 3867, 5955, 4432, 7724, 3244, 15456, 32076, 6187, 2395, 45644, 36348, 17703, 29178, 2379, 100, 2446, 14109, 43252, 35828, 11145, 102], [101, 2118, 15242, 12006, 11391, 5068, 58634, 3420, 19264, 27900, 2392, 2554, 8844, 5602, 2395, 44222, 28550, 95462, 2040, 5333, 10706, 16151, 49988, 5718, 12041, 56774, 21327, 3184, 17436, 2551, 2645, 2076, 9294, 81528, 2508, 102], [101, 100, 1040, 100, 1040, 100, 100, 1040, 100, 1040, 100, 100, 100, 100, 1040, 100, 1040, 100, 19675, 36187, 7428, 12499, 2989, 47907, 98253, 100, 1040, 100, 1040, 100, 102], [101, 100, 62081, 5602, 17618, 2046, 41691, 6187, 14109, 2765, 7043, 12351, 458

In [27]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Map: 100%|██████████| 1003/1003 [00:00<00:00, 14717.32 examples/s]


The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Map: 100%|██████████| 112/112 [00:00<00:00, 12407.22 examples/s]


The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


Map: 100%|██████████| 2601/2601 [00:00<00:00, 21034.11 examples/s]


In [28]:

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list)).to(device)

/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
data_collator = DataCollatorForTokenClassification(tokenizer)

metric = load_metric("seqeval")

labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])


/tmp/ipykernel_1914756/4108205622.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning:

{'ER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'UM': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [30]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [31]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    f"generated-{task}",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=8150,  # Set to None to log based on epochs
    evaluation_strategy='steps',
    save_strategy='steps',
    save_steps=8150,  # Adjust as needed
    logging_first_step=False,  # Ensure logging starts from the first epoch
    num_train_epochs=100  # Set the number of training epochs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()



You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
8150,0.040200,1.251651,0.725302,0.741841,0.733478,0.846843
16300,0.000900,1.494408,0.724173,0.750837,0.737264,0.851674


/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: TIME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: TIME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE 

The OrderedVocab you are attempting to save contains holes for indices [1015, 1016, 1017, 1018, 1053, 1054, 1055, 1056, 1057, 1060, 1061, 1062, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1099, 1101, 1112, 1113, 1556, 1557, 1568], your vocabulary could be corrupted !


TrainOutput(global_step=16300, training_loss=0.02055444843198624, metrics={'train_runtime': 6074.8901, 'train_samples_per_second': 42.816, 'train_steps_per_second': 2.683, 'total_flos': 6818164338394332.0, 'train_loss': 0.02055444843198624, 'epoch': 100.0})

In [40]:
trainer.evaluate()

{'eval_loss': 1.494408130645752,
 'eval_precision': 0.7241727199354318,
 'eval_recall': 0.750836820083682,
 'eval_f1': 0.7372637633525061,
 'eval_accuracy': 0.8516739692169419,
 'eval_runtime': 6.9997,
 'eval_samples_per_second': 143.291,
 'eval_steps_per_second': 9.0,
 'epoch': 100.0}

In [41]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

In [42]:
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_predictions

[['TIME',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'ORG',
  'ORG',
  'ORG',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'PER',
  'PER',
  'PER',
  'PER',
  'LOC',
  'ORG',
  'ORG',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'NUM',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER'],
 ['OBJ',
  'OBJ',
  'OBJ',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'NUM',
  'OBJ',
  'OBJ',
  'ORG',
  'ORG',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER'],
 ['TIME',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER',
  'OBJ',
  'OBJ',
  'OBJ',
  'OBJ',
  'OBJ',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'PER',
  'PER',
  'PER',
  'PER',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC'],
 ['PER',
  'PER',
  'PE

In [43]:
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels

[['TIME',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'ORG',
  'ORG',
  'ORG',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'NUM',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER'],
 ['OBJ',
  'OBJ',
  'OBJ',
  'OBJ',
  'OBJ',
  'LOC',
  'LOC',
  'NUM',
  'ORG',
  'ORG',
  'ORG',
  'ORG',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER'],
 ['TIME',
  'TIME',
  'TIME',
  'TIME',
  'TIME',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER',
  'PER',
  'LOC',
  'OBJ',
  'OBJ',
  'OBJ',
  'OBJ',
  'OBJ',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'PER',
  'PER',
  'PER',
  'PER',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC',
  'LOC'],
 ['PER',
  'PER',
  'OB

In [44]:
results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)

{'ER': {'precision': 0.5041952707856598, 'recall': 0.5339256865912763, 'f1': 0.5186347587289133, 'number': 1238}, 'IME': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1136}, 'OC': {'precision': 0.5886666666666667, 'recall': 0.6056241426611797, 'f1': 0.5970250169033131, 'number': 1458}, 'UM': {'precision': 0.9008919722497523, 'recall': 0.9588607594936709, 'f1': 0.9289729177312214, 'number': 948}, 'overall_precision': 0.7241727199354318, 'overall_recall': 0.750836820083682, 'overall_f1': 0.7372637633525061, 'overall_accuracy': 0.8516739692169419}


In [37]:

# Label Test: 
#  [' B-CORP', ' B-CW', ' B-GRP', ' B-LOC', ' B-PER', ' B-PROD', ' I-CORP', ' I-CW', ' I-GRP', ' I-LOC', ' I-PER', ' I-PROD', ' O', 'NUM]

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from pathlib import Path
import torch
# Define the device if using GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
# Provide the path to the tokenizer and model directory

tokenizer = AutoTokenizer.from_pretrained(Path("/home/bikas/NER_Jsonl/generated-ner/checkpoint-16300"), device=device)
model = AutoModelForTokenClassification.from_pretrained(Path("/home/bikas/NER_Jsonl/generated-ner/checkpoint-16300")).to(device)

# Create NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True, device=device)


# example = """প ৫২২৪৬

# করিয়া তাহতো বা সে মালিক ভোগ দখলকারও দখলকারিন নিয়ত ইয়া ও কযা বিত সে যে
# এস এ জরিপকালে তাহাদের নিজ নিজ নাম শুদ্ধভাবে রেকর্ড ভুক্ত করাইয়া মালিক ভোগ দখলকার ও
# দখলকারিনী নিয়ত হন। অতপর উক্ত আমির আলী বেপারী পরলোক গমন করিলে তত্ত্যাজ্য বিত্ত পুর
# আবদুল আউয়াল পৈত্রিক ওয়ারিশ সুত্রে মালিক ভোগ দখলকার নিয়ত হইয়া ও থাকিয়া তাহার ভোগ দখলের ্

# সুবিধার্থে অনযা্য ওয়ারিশগনের সহিত ঘরোয়া মৌখিক আপোষ বন্টন মতে যাহার যাহার ছাহাম বুঝাইয়া
# দিয়া ও বর্ণিত দাগের সম্পত্তিতে নিজ ছাহাম বুঝিয়া পাইয়া মালিক ভোগ দখলকার নিয়ত থাকাবনথায় বিগত
# ০১/০১/১৯৭১ খরষটন্দ তারিখে নারায়ণগঞ্জ সাব রেজিদ্রি অফিসে রেজিদ্্িৃত ৪নং এককিন্তা সাফ করলা
# তাহাতে খরিদা সুত্রে মালিক হন। অতঃপর উক্ত লাল মোন নেছা ভিন্ন ভিন্ন দলিল মূলে খরিদ করিয়া
# তাহাতে খরিদা সুত্রে মালিকা ভোগ দখলকারিনী নিয়ত হইয়া ও থাকিয়া বিগত রি- সেঃ মেঃ আর এস
# জরিপকালে তাহার নিজ নাম শুদ্ধভাবে রেকর্ড ভুক্ত করাইয়া মালিক ভোগ দখলকারিনী নিয়ত থাকাব্থায়
# পরলোক গমন করিলে তত্্যাজ্য বিস্তে তাহার ওয়ারিশ মোঃ কছর বেপারী মাতার ওয়ারিশ সুত্রে মালিক
# ভোগ দখলকার নিয়ত থাকানস্থায় বিগত ২৪/১২/১৯৭৯ খ্রষটন্দ তারিখে নারায়ণগঞ্জ সাব রেজিষ্্র অফিসে
# রেজিষ্রিকিত ৯৫৯৫নং এককিন্তা সাফ কবলা দলিল মূলে নিশং গ্রাম নিবাসী মোঃ কছুর বেপারীর পুত্র মোঃ নুর
# হোসেন ও মোঃ নুর হোসেন এর স্ত্রী মোসাঃ আনোয়ারা বেগম উক্ত মোঃ কছর বেপারী হইতে খরিদ করিয়া
# তাহাতে খরিদা সুত্রে মালিক ভোগ দখলকার ও দখলকারিনী নিয়ত থাকাবস্থায় বিগত ২২/১২/১৯৯৭ স্রষ্টা
# তারিখে অত্র অফিসে রেজিষ্্িকৃত ৩৮৪৮নং এককিন্তা সাফ কবলা দলিল মূলে নিশং গ্রাম নিবাসী মরহুম মোঃ
# আৰু তালেব বেপারীর পুত্র মোঃ জাকির হোসেন উক্ত মোঃ নুর হোসেন ও মোসাঃ আনোয়ারা বেগম হইতে
# খরিদ করিয়া তাহাতে খরিদা সুত্রে মালিক ভোগ দখলকার নিয়ত থাকাবন্থায় পরলোক গমন করিলে
# তত্ত্যাজা বিত্তে তিনপুত্র তিনকন্যা সাইফুর রহমান, পান্না মিয়া, পারভেজ মিয়া, জাকিয়া বেগম, জেনিফা ও
# জিয়াসমিন পৈত্রিক ওয়ারিশ সুত্রে এবং দুই স্্ী সামসুন নাহার ও আমি অত্র দলিল দাত্রী (পারভীন) স্বামীর
# ওয়ারিশ সুত্রে অতি মরহুম মেগা জাকির হোসেন এর ওয়ারিশ সুত্রে মালিক ভোগ দখলকার ও দখলকারিনী
# নিয়ত হইয়া ও থাকিয়া এজমালিতে ভোগ দখলকার ও দখলকারিনী নিয়ত থাকাবহায় তাহাদের ভোগ
# দখললের সুবিধার্থে বিগত ২৫/০৪/২০১৩ স্রষ্টা তারিখে অত্র অফিসে রেজিদ্রিকৃত ৩০৭০নং এবকিন্তা
# আপোষ বন্টন নামা দলিল মূলে অন্যান্য দাগে যাহার যাহার ছাহাম বুঝাইয়া ও বর্ণিত দাগে আমি অত্র দলিল
# দাতা (পারভিন) নিজ ছাহাম বুঝিয়া পাইয়া মালিকা ভোগ দখলকারিনী নিয়ত হইয়া ও থাকিয়া বিগত
# ২৯/০১/২০১৯ পরষটাব্ তারিখে বন্দর সহকারী কমিশনার (ভূমি) এর বরাবরে নামজারী জমাভাগ মোকদ্দমা
# ২৮১০/১৮-১টনং বলে আর এস খতিয়ান মোতাবেক নিজ নামে নামজারী জমাভাগ করাইয়া নিম তফসিল
# বর্ণিত সম্পত্তিতে প্রকাশ্য ভাবে অন্যের বিনা বাধায় শাঞধিপর্ন ভাবে ভোগ দখল ও চাষাবাদ ইত্যাদি করিয়া
# আসিতেছি। 
# ৯
# £
# """
example = """হলফ নামা ২৫/০৪/২০১৩ রাষ্ট্রপতির ১৯৭২ সনের ১৪২ নং আদেশ, ১৯০৮ সনের রেজিস্ট্রেশন আইনের $২ৎ|]
১৮৮২ সনের সম্পত্তি হস্তান্তর আইনের ী৫| 53& 524(9) এবং অনুসারে প্রদত্ত হলফনামা:
বরাবর, সাব-রেজিট্ট্রার, বাড্ডা সাব-রেজিষ্ট্রা ঢাকা প্রথম পক্ষ শরীফ আহমেদ, জনা
তারিখ- ২৫০০১ ১৯৭০ ইং, জাতীয় পরিচয় পত্র নমবর- ৩৭১৬৮৫১৮৭২, যাহার টিআই-এন নম্বর-
৫৯২১৫৫৪১১৭৭৩, পিতা- মৃত মাতা- হক; আম্বিয়া খানম, ঠিকানা- কামারিয়া ডাকঘর- কামারিয়া
বাজার-২২৫২, থানা- তারাকান্দা, জেলা- ময়মনসিংহ, বর্তমান ঠিকানা- ৭/ক, পন্ডিতপাড়া,
থানা- ময়মনসিংহ-সদর, জেলা- ময়মনসিংহ, ধর্ম_ ইসলাম, জাতীয়তা- বাংলাদেশী, পেশা-
ব্যবসা পর পাতায়- ১২ অফিস, শামছল স্থায়ী পশ্চিম;"""

# Split the text into words
word_list = example.split()
# print(word_list)

# Print the list of words
# print(word_list)


# Get NER results
ner_results = nlp(word_list)
print(ner_results)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[[{'entity_group': 'LABEL_2', 'score': 0.9969043, 'word': 'হলফ', 'start': 0, 'end': 3}], [{'entity_group': 'LABEL_2', 'score': 0.9989209, 'word': 'নামা', 'start': 0, 'end': 4}], [{'entity_group': 'LABEL_5', 'score': 0.9999992, 'word': '২৫ / ০৪ / ২০১৩', 'start': 0, 'end': 10}], [{'entity_group': 'LABEL_4', 'score': 0.9343449, 'word': 'রাষটরপতির', 'start': 0, 'end': 11}], [{'entity_group': 'LABEL_1', 'score': 0.99998343, 'word': '১৯৭২', 'start': 0, 'end': 4}], [{'entity_group': 'LABEL_2', 'score': 0.9353715, 'word': 'সনের', 'start': 0, 'end': 4}], [{'entity_group': 'LABEL_1', 'score': 0.99998343, 'word': '১৪২', 'start': 0, 'end': 3}], [{'entity_group': 'LABEL_0', 'score': 0.9993061, 'word': 'নং', 'start': 0, 'end': 2}], [{'entity_group': 'LABEL_2', 'score': 0.9929844, 'word': 'আদেশ,', 'start': 0, 'end': 5}], [{'entity_group': 'LABEL_1', 'score': 0.99998343, 'word': '১৯০৮', 'start': 0, 'end': 4}], [{'entity_group': 'LABEL_2', 'score': 0.9353715, 'word': 'সনের', 'start': 0, 'end': 4}], [{'

In [5]:
# Iterate through each word and its corresponding NER result
for word, result in zip(word_list, ner_results):
    # Print the word and its NER result
    print(f"Main Word: {word}, NER Result: {result}")


Main Word: হলফ, NER Result: [{'entity_group': 'LABEL_2', 'score': 0.9969043, 'word': 'হলফ', 'start': 0, 'end': 3}]
Main Word: নামা, NER Result: [{'entity_group': 'LABEL_2', 'score': 0.9989209, 'word': 'নামা', 'start': 0, 'end': 4}]
Main Word: ২৫/০৪/২০১৩, NER Result: [{'entity_group': 'LABEL_5', 'score': 0.9999992, 'word': '২৫ / ০৪ / ২০১৩', 'start': 0, 'end': 10}]
Main Word: রাষ্ট্রপতির, NER Result: [{'entity_group': 'LABEL_4', 'score': 0.9343449, 'word': 'রাষটরপতির', 'start': 0, 'end': 11}]
Main Word: ১৯৭২, NER Result: [{'entity_group': 'LABEL_1', 'score': 0.99998343, 'word': '১৯৭২', 'start': 0, 'end': 4}]
Main Word: সনের, NER Result: [{'entity_group': 'LABEL_2', 'score': 0.9353715, 'word': 'সনের', 'start': 0, 'end': 4}]
Main Word: ১৪২, NER Result: [{'entity_group': 'LABEL_1', 'score': 0.99998343, 'word': '১৪২', 'start': 0, 'end': 3}]
Main Word: নং, NER Result: [{'entity_group': 'LABEL_0', 'score': 0.9993061, 'word': 'নং', 'start': 0, 'end': 2}]
Main Word: আদেশ,, NER Result: [{'entity_

In [9]:
def convert_label(label):
    if label == 'LABEL_0':
        return 'LOC'
    elif label == 'LABEL_1':
        return 'NUM'
    elif label == 'LABEL_2':
        return 'OBJ'
    elif label == 'LABEL_3':
        return 'ORG'
    elif label == 'LABEL_4':
        return 'PER'
    elif label == 'LABEL_5':
        return 'TIME'

# Iterate through each word and its corresponding NER result
for word, result in zip(word_list, ner_results):
    # Check if NER result is empty
    if result:
        # Extracting the NER label from the result
        ner_label = result[0]['entity_group']
        # Convert NER label to its corresponding category
        ner_category = convert_label(ner_label)
        # Print the word and its NER category
        print(f"{ner_category} ---> {word}")
    else:
        # Print message for empty NER result
        print(f"No NER result found for: {word}")


OBJ ---> হলফ
OBJ ---> নামা
TIME ---> ২৫/০৪/২০১৩
PER ---> রাষ্ট্রপতির
NUM ---> ১৯৭২
OBJ ---> সনের
NUM ---> ১৪২
LOC ---> নং
OBJ ---> আদেশ,
NUM ---> ১৯০৮
OBJ ---> সনের
OBJ ---> রেজিস্ট্রেশন
OBJ ---> আইনের
NUM ---> $২ৎ|]
NUM ---> ১৮৮২
OBJ ---> সনের
OBJ ---> সম্পত্তি
OBJ ---> হস্তান্তর
OBJ ---> আইনের
NUM ---> ী৫|
NUM ---> 53&
NUM ---> 524(9)
PER ---> এবং
OBJ ---> অনুসারে
PER ---> প্রদত্ত
LOC ---> হলফনামা:
LOC ---> বরাবর,
ORG ---> সাব-রেজিট্ট্রার,
LOC ---> বাড্ডা
PER ---> সাব-রেজিষ্ট্রা
LOC ---> ঢাকা
LOC ---> প্রথম
OBJ ---> পক্ষ
LOC ---> শরীফ
PER ---> আহমেদ,
OBJ ---> জনা
OBJ ---> তারিখ-
NUM ---> ২৫০০১
NUM ---> ১৯৭০
LOC ---> ইং,
ORG ---> জাতীয়
LOC ---> পরিচয়
OBJ ---> পত্র
LOC ---> নমবর-
NUM ---> ৩৭১৬৮৫১৮৭২,
ORG ---> যাহার
ORG ---> টিআই-এন
LOC ---> নম্বর-
NUM ---> ৫৯২১৫৫৪১১৭৭৩,
PER ---> পিতা-
OBJ ---> মৃত
PER ---> মাতা-
PER ---> হক;
LOC ---> আম্বিয়া
PER ---> খানম,
OBJ ---> ঠিকানা-
LOC ---> কামারিয়া
LOC ---> ডাকঘর-
LOC ---> কামারিয়া
LOC ---> বাজার-২২৫২,
LOC ---> থানা-
LOC ---> তারাকান্দা,
LOC

In [10]:
def convert_label(label):
    if label == 'LABEL_0':
        return 'LOC'
    elif label == 'LABEL_1':
        return 'NUM'
    elif label == 'LABEL_2':
        return 'OBJ'
    elif label == 'LABEL_3':
        return 'ORG'
    elif label == 'LABEL_4':
        return 'PER'
    elif label == 'LABEL_5':
        return 'TIME'

# Iterate through each word and its corresponding NER result
for word, result in zip(word_list, ner_results):
    # Check if NER result is empty
    if result:
        # Extracting the NER label from the result
        ner_label = result[0]['entity_group']
        # Check if the label is 'LABEL_1' or 'LABEL_5'
        if ner_label == 'LABEL_1' or ner_label == 'LABEL_5':
            # Convert NER label to its corresponding category
            ner_category = convert_label(ner_label)
            # Print the word and its NER category
            print(f"{ner_category} ---> {word}")
    else:
        # Print message for empty NER result
        print(f"No NER result found for: {word}")


TIME ---> ২৫/০৪/২০১৩
NUM ---> ১৯৭২
NUM ---> ১৪২
NUM ---> ১৯০৮
NUM ---> $২ৎ|]
NUM ---> ১৮৮২
NUM ---> ী৫|
NUM ---> 53&
NUM ---> 524(9)
NUM ---> ২৫০০১
NUM ---> ১৯৭০
NUM ---> ৩৭১৬৮৫১৮৭২,
NUM ---> ৫৯২১৫৫৪১১৭৭৩,
TIME ---> ৭/ক,
NUM ---> ১২


In [32]:
# Initialize an empty list to store entities with LABEL_2 and LABEL_5 along with their index positions
filtered_entities_with_index = []

# Iterate through the NER results along with their index positions
for index, entity_list in enumerate(ner_results):

    # Check if the entity list is not empty
    if entity_list:
        # Extract the label and the text of the entity
        label = entity_list[0].get('entity_group', '')  # Use .get() to avoid KeyError
        text = entity_list[0].get('word', '')  # Use .get() to avoid KeyError

        # Check if the label is LABEL_1 or LABEL_5
        if label == 'LABEL_0' or label == 'LABEL_1' or label == 'LABEL_2' or label == 'LABEL_3' or label == 'LABEL_4' or label == 'LABEL_5':
            # Append the entity along with its index position to the filtered list
            filtered_entities_with_index.append((index, label, text))

# Define a function to convert labels to desired format
def convert_label(label):
    if label == 'LABEL_0':
        return 'LOC'
    elif label == 'LABEL_1':
        return 'NUM'
    elif label == 'LABEL_2':
        return 'OBJ'
    elif label == 'LABEL_3':
        return 'ORG'
    elif label == 'LABEL_4':
        return 'PER'
    elif label == 'LABEL_5':
        return 'TIME'
    else:
        return 'OTHER'
# Label Test: 
#  ['LOC', 'NUM', 'OBJ', 'ORG', 'PER', 'TIME']
# Print the filtered entities along with their index positions
for index, label, text in filtered_entities_with_index:
    converted_label = convert_label(label)
    print(f'{converted_label} ---> {text}')


LOC ---> প
NUM ---> ৫২২৪৬
LOC ---> করিযা
OBJ ---> তাহতো
OBJ ---> বা
OBJ ---> সে
PER ---> মালিক
LOC ---> ভোগ
LOC ---> দখলকারও
LOC ---> দখলকারিন
OBJ ---> নিযত
LOC ---> ইযা
LOC ---> ও
LOC ---> কযা
LOC ---> বিত
OBJ ---> সে
OBJ ---> যে
OBJ ---> এস
ORG ---> এ
LOC ---> জরিপকালে
OBJ ---> তাহাদের
OBJ ---> নিজ
OBJ ---> নিজ
OBJ ---> নাম
OBJ ---> শদধভাবে
OBJ ---> রেকরড
LOC ---> ভকত
LOC ---> করাইযা
PER ---> মালিক
LOC ---> ভোগ
OBJ ---> দখলকার
LOC ---> ও
OBJ ---> দখলকারিনী
OBJ ---> নিযত
PER ---> হন
ORG ---> অতপর
LOC ---> উকত
PER ---> আমির
PER ---> আলী
PER ---> বেপারী
OBJ ---> পরলোক
OBJ ---> গমন
ORG ---> করিলে
LOC ---> তততযাজয
OBJ ---> বিতত
OBJ ---> পর
PER ---> আবদল
LOC ---> আউযাল
LOC ---> পৈতরিক
LOC ---> ওযারিশ
OBJ ---> সতরে
PER ---> মালিক
LOC ---> ভোগ
OBJ ---> দখলকার
OBJ ---> নিযত
LOC ---> হইযা
LOC ---> ও
LOC ---> থাকিযা
OBJ ---> তাহার
LOC ---> ভোগ
LOC ---> দখলের
OBJ ---> সবিধারথে
LOC ---> অনযায
LOC ---> ওযারিশগনের
LOC ---> সহিত
LOC ---> ঘরোযা
NUM ---> মৌখিক
LOC ---> আপোষ
LOC ---> বনটন
OBJ -

In [ ]:
# হলফ নামা রাষ্ট্রপতির ১৯৭২ সনের ১৪২ নং আদেশ, ১৯০৮ সনের রেজিস্ট্রেশন আইনের $২ৎ|]
# ১৮৮২ সনের সম্পত্তি হস্তান্তর আইনের ী৫| 53& 524(9) এবং অনুসারে প্রদত্ত হলফনামা:
# বরাবর, সাব-রেজিট্ট্রার, বাড্ডা সাব-রেজিষ্ট্রা ঢাকা প্রথম পক্ষ শরীফ আহমেদ, জনা
# তারিখ- ২৫০০১ ১৯৭০ ইং, জাতীয় পরিচয় পত্র নমবর- ৩৭১৬৮৫১৮৭২, যাহার টিআই-এন নম্বর-
# ৫৯২১৫৫৪১১৭৭৩, পিতা- মৃত মাতা- হক; আম্বিয়া খানম, ঠিকানা- কামারিয়া ডাকঘর- কামারিয়া
# বাজার-২২৫২, থানা- তারাকান্দা, জেলা- ময়মনসিংহ, বর্তমান ঠিকানা- ৭/ক, পন্ডিতপাড়া,
# থানা- ময়মনসিংহ-সদর, জেলা- ময়মনসিংহ, ধর্ম_ ইসলাম, জাতীয়তা- বাংলাদেশী, পেশা-
# ব্যবসা পর পাতায়- ১২ অফিস, শামছল স্থায়ী পশ্চিম;
# # Label Test: 
# #  ['LOC', 'NUM', 'OBJ', 'ORG', 'PER', 'TIME']

In [59]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from pathlib import Path
import torch
# Define the device if using GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
# Provide the path to the tokenizer and model directory
tokenizer = AutoTokenizer.from_pretrained(Path("/home/bikas/NER_Jsonl/model100-ner/checkpoint-14100"))
model = AutoModelForTokenClassification.from_pretrained(Path("/home/bikas/NER_Jsonl/model100-ner/checkpoint-14100")).to(device)

# Move model to the specified device
model.to(device)

# Create NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True, device=device)



# # Initialize pipeline
# pipeline = TokenClassificationPipeline(tokenizer=tokenizer, model=model)

# raw_text = "এই ইউনিয়নে ৩ টি মৌজা ও ১০ টি গ্রাম আছে ।" # Change me
# output = pipeline(raw_text)

# Example text
# example = """প ৫২২৪৬
# ০১/০১/১৯৭১ খরষটন্দ 78 তারিখে নারায়ণগঞ্জ সাব রেজিদ্রি অফিসে রেজিদ্্িৃত ৪নং এককিন্তা সাফ করলা
# তাহাতে খরিদা সুত্রে মালিক হন। অতঃপর উক্ত লাল মোন নেছা ভিন্ন ভিন্ন দলিল মূলে খরিদ করিয়া
# তাহাতে খরিদা সুত্রে মালিকা ভোগ দখলকারিনী নিয়ত হইয়া ও থাকিয়া বিগত রি- সেঃ মেঃ আর এস
# জরিপকালে তাহার নিজ নাম শুদ্ধভাবে রেকর্ড ভুক্ত করাইয়া মালিক ভোগ দখলকারিনী নিয়ত থাকাব্থায়
# পরলোক গমন করিলে তত্্যাজ্য বিস্তে তাহার ওয়ারিশ মোঃ কছর বেপারী মাতার ওয়ারিশ সুত্রে মালিক
# ভোগ দখলকার নিয়ত থাকানস্থায় বিগত ২৪/১২/১৯৭৯ খ্রষটন্দ তারিখে নারায়ণগঞ্জ 45 সাব রেজিষ্্র অফিসে রেজিষ্রিকিত 
# ৯৫৯৫নং এককিন্তা সাফ কবলা দলিল মূলে নিশং গ্রাম নিবাসী মোঃ কছুর বেপারীর পুত্র মোঃ নুর
# হোসেন ও মোঃ নুর হোসেন এর স্ত্রী মোসাঃ আনোয়ারা বেগম উক্ত মোঃ কছর বেপারী হইতে খরিদ করিয়া
# """
example = ['৫২২৪৬', '০১/০১/১৯৭১', 'ইউনিয়নে', 'মৌজা', 'নারায়ণগঞ্জ']
# Get NER results
ner_results = nlp(example)
print(ner_results)


/home/bikas/miniconda3/envs/venv/lib/python3.9/site-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[[{'entity_group': 'LABEL_13', 'score': 0.9158354, 'word': '৫২২৪৬', 'start': 0, 'end': 5}], [{'entity_group': 'LABEL_13', 'score': 0.9996762, 'word': '০১ / ০১ / ১৯৭১', 'start': 0, 'end': 10}], [{'entity_group': 'LABEL_8', 'score': 0.9393903, 'word': 'ইউনি', 'start': 0, 'end': 4}, {'entity_group': 'LABEL_12', 'score': 0.6029293, 'word': '##যনে', 'start': 4, 'end': 8}], [{'entity_group': 'LABEL_13', 'score': 0.9158354, 'word': 'মৌজা', 'start': 0, 'end': 4}], [{'entity_group': 'LABEL_12', 'score': 0.9984384, 'word': 'নারাযণগঞজ', 'start': 0, 'end': 11}]]
